<a href="https://colab.research.google.com/github/Mikael-lab/Proyecto-final-codigo-facilito/blob/main/Analisis_cuentas_banamex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Exploración de datos de créditos Bancarios en su proceso de cobranza en el año 2010

Bienvenido, en este proyecto utiliza información obtenida de un proceso de cobranza.


*Cobranza*:
La cobranza es el acto o procedimiento por el cual se consigue la contraprestación por un bien o servicio o la cancelación de una deuda.

Referencia: https://economipedia.com/definiciones/cobranza.html


Para este proyecto nos enfocaremos en analizar la información de contacto con el deudor, dicho de otra manera, la gestión de la cuenta.


El objetivo final de cada gestión realizada a una cuenta es conseguir una promesa de pago por parte del titular.

Una promesa de pago es el registro de la fecha en que el titular se compromete a realizar el pago solicitado.

Dados estos escenarios, las preguntas que buscamos responder son las siguientes:

1. ¿Que cantidad de Gestiones se realizan en periodos de 1 mes?

2. ¿Que cantidad de promesas de pago se consiguieron en periodos de 1 mes?

3. ¿Hay relación entre el volumen de Gestiones y la cantidad de promesas de pago?


## Carga de datos originales

In [8]:
#Utilizaremos la libreria de pandas para manipular los datos originales
import pandas as pd


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Proyecto final Data Science/gestiones_bnx_filtradas.csv')

In [3]:
# Damos un primer vistazo a nuestros datos
data

,IdGestion,IdCtaDesp,Fecha,Hora,IdTipoGestion,IdCodAccion,IdCodResultado,TipoPP,Monto,Comentarios
0,8081195,905361,2010-10-07,16:31:30,1,33,36,0.0,0.0,MENSAJE BUZON
1,8082138,915998,2010-10-07,18:34:57,1,33,42,0.0,0.0,240 NO REG TELEF DE TT
2,8082137,662872,2010-10-07,18:31:54,1,33,50,0.0,0.0,NO CONTESTA SIN OPCIONA BUZON
3,8082136,910303,2010-10-07,18:36:35,1,33,302,0.0,0.0,83595998 NUM FUERA DE SERVICIO
4,8082135,195590,2010-10-07,18:35:55,1,33,41,0.0,0.0,TEL DA TONO DE OCUPADO
...,...,...,...,...,...,...,...,...,...,...
1028384,8528500,230090,2010-12-27,19:47:00,3,38,27,0.0,0.0,SE DEJO NOTIFICACION CON TITULAR INFORMA PAGAR...
1028385,8528501,230090,2010-12-30,10:01:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON VECINO DE TITULAR DE ...
1028386,8528502,261142,2010-12-27,20:49:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON INQUILINO DE TITULAR ...
1028387,8528503,261142,2010-12-30,12:31:00,3,38,73,0.0,0.0,SE DEJO NOTIFICACION CON INQUILINO DE TITULAR ...


## Propiedades del conjunto de datos

In [9]:
# Para este punto vamos a utilizar la libreria funpymodelin
!pip install funpymodeling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached funpymodeling-0.1.7-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 KB 5.6 MB/s eta 0:00:00
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
from funpymodeling.exploratory import status

ModuleNotFoundError: ignored

In [ ]:
# q_nan = cantidad de nulos
# p_nan = porcentaje que representan los valores nulos
# q_zeros = cantidad de ceros
# p_zeros = porcentaje que representan los valores 0
# unique = cantidad de valores unicos
# type, object = string
status(data)

Nuestro archivo de datos original contiene un tota de 10 columnas, para responder las interrogantes definidas en el proyecto solamente utilizaremos IdCodResultado, Fecha.

Observando la tabla de propiedades podemos saber que los campos que utilizaremos no tienen registros con valores nulos, esta se consiguio desde las reglas definidas en el proceso de cobranza:
- Toda gestión debe tener un resultado.
- Toda gestión debe tener una fecha en que se realizo.

In [ ]:
# Creamos un nuevo data frame que contenga únicamente las 2 columnas que utilizaremos
df_fecha_codResultado = data[['Fecha','IdCodResultado']]

In [ ]:
df_fecha_codResultado

In [ ]:
# Para poder gráficas nuestro dataFrame vamos a convertir los campos
# Fecha lo convertimos a DateTime
# IdCodResultado lo convertimos a String

df_fecha_codResultado.Fecha = pd.to_datetime(df_fecha_codResultado.Fecha,dayfirst=True)
df_fecha_codResultado.IdCodResultado = df_fecha_codResultado['IdCodResultado'].astype('string')

# Realizamos un crosstab y lo almacenamos en un nuevo dataFrame
df_fecha_coResultado_group = pd.crosstab(df_fecha_codResultado.Fecha, df_fecha_codResultado.IdCodResultado).resample("M").sum().reset_index()

df_fecha_coResultado_group


# Visualización de datos

In [ ]:
# Para la visualización de datos vamos a utilizar la librería Altair
import altair as alt

In [ ]:
# Promesas de pago por mes
alt.Chart(df_fecha_coResultado_group).mark_bar(size=60).encode(
    y= alt.Y("49", title="Promesas"),
    x= alt.X("Fecha", title = "",timeUnit='month'),
    color = alt.Color('49', title = "Volumen", scale = alt.Scale( scheme = "blues")),
    tooltip=['49']
).interactive().properties(
    title="Promesas de pago en el segundo semestre del año 2010"
)

Con el trabajo que hemos realizado hasta este punto ya pudimos identificar **el mes en el que se registraron más promesas de pago, que fue Agosto con un total de 5,402**.

El paso siguiente es saber el volumen de gestiones totales en comparacion con la cantidad de promesas de pago

In [ ]:
# Obtenemos los totales por mes de todos los codigos de resultado
# con la función crosstab hacemos el cruce para tener como columnas los IdCodResultado
# la función resample nos ayuda a poner como fila el mes
# la función sum para hacer la sumatoria de las veces que aparece un IdCodResultado

df_totales = pd.crosstab(df_fecha_codResultado.Fecha,df_fecha_codResultado.IdCodResultado).resample("M").sum().reset_index()

# Creamos la columna y fila del total de IdCodResultado que sería equivalente a contar las gestiones
df_totales.loc['total', :] = df_totales.sum(numeric_only=True)
df_totales['total'] = df_totales.sum(axis=1, numeric_only=True)
df_totales


In [ ]:
# Creamos los objetos para graficar
scale = alt.Scale(domain=["Promesas","Gestiones"], range=['red','lightblue'])

base = alt.Chart(df_totales).transform_calculate(
    line="'Promesas'",
    bar="'Gestiones'"
).encode(
    alt.X("Fecha",title="",timeUnit='month'),
).properties(
    title="Volumen de gestiones y cantidad de promesas conseguidas"
)

bar = base.mark_bar().encode(
    alt.Y('total',title="Gestiones"),
    tooltip=[alt.Tooltip("total", title='Gestiones')],
    color=alt.Color('bar:N', scale=scale, title=''),
)

line = base.mark_line(point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y("49",title=""),
    tooltip=[alt.Tooltip('49',title='Promesas')],
    color=alt.Color('line:N', scale=scale, title='')
)

text_line = line.mark_text(
    align='right',
    baseline='bottom',
    dx=3
).encode( 
    text='49'
)

bar+(line+text_line).interactive()


Con la siguiente gráfica podemos observar lo siguiente:

1. El volumen de gestiones es abismalmente superior a la cantidad de promesas de pago generadas en cada mes.

2. El volumen de gestiones no siempre es proporcional a la cantidad de promesas de pago obtenidas. 
En el mes de Septiembre se obtuvieron un total de 3,952 promesas de pago con un volumen de gestiones de 217,595, en el mes de Octubre se obtuvieron 4,296 con un volumen de 192,522.

# Conclusiones
En este punto ya podemos contestar las preguntas que planteamos al principio.

1. ¿Que cantidad de Gestiones se realizan en periodos de 1 mes?
2. ¿Que cantidad de promesas de pago se consiguieron en periodos de 1 mes?
**R= se puede visualizar en la gráfica "Volumen de gestiones y cantidad de promesas conseguidas"**

3. ¿Hay relación entre el volumen de Gestiones y la cantidad de promesas de pago? 
**R= con los datos que se tienen hasta el momento no se observa una relación que nos indique que a mayor volumen de gestiones mayor cantidad de promesas de pago se van a obtener, claro que esto es forma general.**

# Segunda parte

Con el análisis anterior, surgieron nuevas preguntas las cuales podrían esclarecer la manera en que se trata a una cuenta.

Las preguntas que surgieron fueron las siguientes:

1. ¿Que cantidad de gestiones recibe en promedio una cuenta?

1. ¿En periodos de 1 mes se llega a gestionar todas las cuentas?

In [ ]:
# Creamos una copia del dataframe original
df_copy = data
# Convertimos el campo a datetime
df_copy.Fecha = pd.to_datetime(df_copy.Fecha,dayfirst=True)

# Convertimos a entero el la fecha y la insertamos en una nueva columna llamada Mes
df_copy.insert(0,'Mes', (data['Fecha'].dt.month).astype(int) )





In [ ]:
# Agrupamos primero por mes y despues por IdCtaDesp para no hacer un conteo doble de gestiones
df_gestiones_promedio = df_copy.groupby(['Mes','IdCtaDesp'],as_index=False).IdCtaDesp.count()

df_gestiones_promedio


In [ ]:
# Agrupamos por el mes, agregando el promedio del campo IdCTaDesp que contiene el total de gestiones y los redondeamos
df_promedio_final = df_gestiones_promedio.groupby(['Mes'],as_index=False).mean('IdCtaDesp').round()

# Convertimos el mes a texto para mejor visualización al momento de generar el gráfico
df_promedio_final['Mes'] = pd.to_datetime(df_promedio_final['Mes'], format='%m').dt.strftime('%b')

df_promedio_final =df_promedio_final.sort_index()

df_promedio_final


In [ ]:
# Promedio de gestiones realizadas a una cuenta por mes
 
c1 = alt.Chart(df_promedio_final).mark_bar().encode(
    y= alt.Y("IdCtaDesp", title="Gestiones"),
    x= alt.X("Mes", title = "",sort=['Mes']),
    color = alt.Color('IdCtaDesp', title = "Gestiones", scale = alt.Scale( scheme = "blues")),
    tooltip=[alt.Tooltip('IdCtaDesp',title="Gestiones")]
).interactive().properties(
    title="Promedio de gestiones por cuenta",
    width = 350
)

line = base.mark_line(point=alt.OverlayMarkDef(color="red")).encode(
    alt.Y("49",title=""),
    tooltip=[alt.Tooltip('49',title='Promesas')],
    color=alt.Color('line:N', scale=scale, title='')
)

text_line = line.mark_text(
    align='right',
    baseline='bottom',
    dx=3
).encode( 
    text='49'
)

c2 = line+text_line

c1 | c2

Obteniendo el promedio de gestiones que recibe una cuenta en un mes, podemos observar que va en aumento, sin embargo las promesas obtenidas no son proporcionales, lo que pudiera indicarnos `que la insitencia en la cobranza no siempre es la clave para conseguir una promesa de pago`.

In [ ]:
# En el dataFrame df_gestiones_promedio ya teniamos agrupadas las cuentas, entonces se eliminaron los duplicados
# por lo cual vamos a hacer el conteo en vez del promedio para saber que cantidad de cuentas se gestionan en periodos de 1 mes
df_ctas_mes = df_gestiones_promedio.groupby(['Mes'],as_index=False).count()

df_ctas_mes['Mes'] = pd.to_datetime(df_ctas_mes['Mes'], format='%m').dt.strftime('%b')

df_ctas_mes


In [ ]:
# importamos spacing de numpy para la separación de gráficas
from numpy import spacing

c1 = alt.Chart(df_ctas_mes).mark_bar().encode(
    y= alt.Y("IdCtaDesp", title="Cuentas"),
    x= alt.X("Mes", title = "",sort=['Mes']),
    color = alt.Color('IdCtaDesp', title = "Cuentas", scale = alt.Scale( scheme = "blues")),
    tooltip=[alt.Tooltip('IdCtaDesp',title="Cuentas")]
).interactive().properties(
    title="Total de Cuentas Gestionadas por mes",
    width=500
)

c2 = alt.Chart(df_totales).mark_bar().encode(
    alt.X("Fecha",title="",timeUnit='month'),
    alt.Y('total',title="Gestiones"),
    color = alt.Color('total', title = "Gestiones", scale = alt.Scale( scheme = "teals")),
).properties(
    title="Volumen de Gestiones por mes",
    
)

combined = alt.hconcat(
    c1, c2
).resolve_scale(
    color='independent'
).properties(
    spacing=100
)

combined

Con las gráficas anteriores podemos ver que la cantidad de cuentas gestionadas decrementa al igual que el volumen de gestiones en cada uno de los periodos, entonces empezamos a tener `menos cuentas gestionadas, menos volumen de gestiones y el promedio de gestiones por cuenta aumenta.`


# Conclusiones
Las preguntas que surgieron fueron las siguientes:

¿Que cantidad de gestiones recibe en promedio una cuenta?
**R= rango de 2 a 7 en el semestre**
¿En periodos de 1 mes se llega a gestionar todas las cuentas?
**R= Únicamente en el mes de Agosto se gestionaron todas las cuentas

Para complementar las respuestas a las interrogantes se deben mencionar escenarios que pasan en la operacion de la agencia de cobranza, que son los siguientes:


- **Disminución de cuentas a gestionar:** esta disminución de cuentas puede darse por que existen cuentas que su deuda puede quedar liquidada, al entrar en este estado ya no se contempla dentro del universo de cuentas a gestionar.

- **Cantidad de gestiones por cuenta:** existen cuentas en las cuales obtienes una promesa de pago a la primer gestion, hay cuentas que hasta la *n* gestion obtienes una promesa de pago, estas gestiones previas a una promesa de pago se quedan registradas para que el la persona que interactue con el titular de la cuenta sepa el proceso de gestión de la misma.